In [4]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.llms import Ollama


In [58]:
def parse_response(llm_response):
    return llm_response.split("{")[-1].split("}")[0].split("answer : ")[-1].split("\n")[0]
    

In [73]:
question = "Variant rs2476601 does not associate with Rheumatoid Arthritis"


In [74]:
VECTOR_DB_PATH = "../../../data/vectorDB/disease_context_chromaDB_using_all_MiniLM_L6_v2_sentence_transformer_model_with_chunk_size_3000"
SENTENCE_EMBEDDING_MODEL = "all-MiniLM-L6-v2"

RETRIEVAL_SCORE_THRESH = 0.7


In [75]:
embedding_function = SentenceTransformerEmbeddings(model_name=SENTENCE_EMBEDDING_MODEL)

vectorstore = Chroma(persist_directory=VECTOR_DB_PATH, 
                     embedding_function=embedding_function)


In [76]:

# llm = Ollama(base_url="http://localhost:11434",
#              model="llama2:13b",
#              temperature=0,
#              verbose=True,
#              callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

llm = Ollama(base_url="http://localhost:11434",
             model="llama2:13b",
             temperature=0,
             verbose=True)


In [77]:
search_result = vectorstore.similarity_search_with_score(question, k=10000)
score_range = (search_result[-1][-1] - search_result[0][-1]) / (search_result[-1][-1] + search_result[0][-1])
thresh = RETRIEVAL_SCORE_THRESH*score_range
retrieved_context = ""
for item in search_result:
    item_score = (search_result[-1][-1] - item[-1]) / (search_result[-1][-1] + item[-1])
    if item_score < thresh:
        break
    retrieved_context += item[0].page_content
    retrieved_context += "\n"
    

In [78]:
prompt = """
Use the following pieces of context to answer the question at the end as True or False or Don't know. 
Context: {}
Question : {}
Based on the context provided, return the answer in the following format:
{{
answer : <True> or <False> or <Don't know>
}}
""".format(retrieved_context, question)


In [79]:
%%time

llm_response = llm(prompt)


CPU times: user 102 ms, sys: 78.9 ms, total: 181 ms
Wall time: 2min 24s


In [80]:
parse_response(llm_response)


'False'